In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Make sure that caffe is on the python path:
caffe_root = '../../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

../../python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Net<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
../../python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Blob<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
../../python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Solver<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \


In [5]:
caffe.set_mode_cpu()
#net = caffe.Net('../../models/swimmers_fullconv/swimmers_fullconv_big_51x51.prototxt', 
#                '../../models/swimmers_fullconv/swimmers_fullconv_big_51x51.protobuf', caffe.TEST)
net = caffe.Net('../../models/swimmers_fullconv/swimmers_fullconv.prototxt', 
                '../../models/swimmers_fullconv/swimmers_fullconv.caffemodel', caffe.TEST)
net.reshape()

In [6]:
# find max_kernels weight blob
app_weights = net.params['inference'][0].data
idpr_weights = net.params['inference'][1].data
inf_weights = net.params['inference'][2].data
# max_kernels.reshape(1, 152, 35, 35)
print app_weights.shape
print app_weights
print idpr_weights.shape
print idpr_weights
print inf_weights.shape
print inf_weights

(13,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
(152,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]
(12, 915)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [7]:
weightsfile = '/data/hdd/harzig/idpr/idpr_code/weights.txt'


def getW(fname):

    with open(fname) as f:
        content = f.readlines()
        
    nums = content[0].split("\t")
    nbApp = int(nums[0])
    nbIdpr = int(nums[1])
    nbDef = int(nums[2])
    nbBias = int(nums[3])

    #print nbApp, nbIdpr, nbDef, nbBias
    arr = content[1].split("\t")[:-1]
    #print (len(arr))
    w = map(float, arr)
    
    return nbApp, nbIdpr, nbDef, nbBias, w
    

def getDefWeight(fname, idx):
    nbApp, nbIdpr, nbDef, nbBias, w = getW(fname)
    return w[nbApp+nbIdpr+2*idx]
    

def getAppWeight(fname, idx):
    nbApp, nbIdpr, nbDef, nbBias, w = getW(fname)
    return w[idx]
    

def getIdprWeight(fname, idx):
    nbApp, nbIdpr, nbDef, nbBias, w = getW(fname)
    return w[nbApp + idx]
    

In [8]:
getDefWeight(weightsfile, 0)

0.000145480002192

In [9]:
# app weights
for i in xrange(len(app_weights)):
    app_weights[i] = getAppWeight(weightsfile, i)

In [10]:
# idpr weights
for i in xrange(len(idpr_weights)):
    idpr_weights[i] = getIdprWeight(weightsfile, i)

In [11]:
print app_weights, idpr_weights

[ 1.09621286  0.95296675  0.98521286  1.04119349  0.90531331  0.93934429
  0.97585994  0.93734163  0.90461165  1.01195228  1.06271768  1.09141815
  0.94990855] [ 1.03209507  0.92009884  1.00095177  0.98417425  0.97996563  1.09713984
  1.05034983  0.99681979  1.01559186  1.02080476  1.04906559  1.07153773
  1.00231814  1.08399558  1.05299282  0.91242748  0.94260001  1.06054032
  1.04545438  1.00901711  0.93873203  0.95100695  1.05515635  1.09075558
  0.95878881  0.92326629  1.06175244  0.93323022  1.01065707  0.90047115
  1.09944689  0.90231824  0.95849252  1.02305615  0.96060085  0.99355119
  1.0622015   1.00351357  1.03879714  1.0129869   1.0973655   0.95312589
  1.09404159  0.95652074  0.91883159  1.00791276  0.92635107  1.07937729
  0.97353244  0.99549675  1.00919902  0.92269731  0.98536545  0.94859672
  0.90581608  1.0771296   0.91503638  1.0764637   0.96241409  1.00676894
  1.00686538  0.91978419  1.0148325   1.0109576   1.08593464  1.09127724
  1.05205524  1.05204427  1.09879935 

In [12]:
# load connections and which maps to add up
inferencefile = '/data/hdd/harzig/idpr/idpr_code_dcnn/inference.txt'

def getI(fname):
    

    arr = []
    with open(fname) as f:
        for content in f:    
            nums = content.split(" ")

            connectionblob = []

            child = int(nums[0])
            connectionblob.append(child)
            parent = int(nums[1])
            connectionblob.append(parent)
            num_idprmaps = int(nums[2])    
            connectionblob.append(num_idprmaps)

            childmaps = []
            parentmaps = []

            for i in xrange(2 *num_idprmaps):
                connectionblob.append(int(nums[3+i]))

            arr.append(connectionblob)
    
    return arr
    

        
        #childmaps.append(nums[3 + i])
        #parentmaps.append(nums[3 + num_idprmaps + i])
    
    
    
    #print nbApp, nbIdpr, nbDef, nbBias
    #arr = content[1].split("\t")[:-1]
    #print (len(arr))
    #w = map(float, arr)
    
    #return nbApp, nbIdpr, nbDef, nbBias, w

In [13]:
modelfile = '/data/hdd/harzig/idpr/idpr_code_dcnn/model.txt'
out = []
with open(modelfile) as f:
    l = f.readlines()
    line = 0
    nbjoints = int(l[line])
    line += 1
    for i in xrange(nbjoints):
        nbedge = int(l[line])
        line += 1
        for j in xrange(nbedge):
            fromto = l[line].split("\t")
            line += 1
            from_ = int(fromto[0])
            to_ = int(fromto[1])
            if from_ == -1 or to_ == -1:
                continue
            nbcluster = int(l[line])
            print nbcluster, from_, to_
            line += 1
            coordsarr = []
            for c in xrange(nbcluster):                
                coords = l[line].split("\t")
                line += 1
                x = float(coords[0])
                y = float(coords[1])
                coordsarr.extend([x, y])
            
            out.append([from_, to_, coordsarr]) 

3 0 1
3 0 7
3 1 0
10 1 2
3 1 4
10 2 1
10 2 3
10 3 2
3 4 1
6 4 5
6 5 4
6 5 6
6 6 5
3 7 0
10 7 8
3 7 10
10 8 7
10 8 9
10 9 8
3 10 7
6 10 11
6 11 10
6 11 12
6 12 11


In [14]:
t = getI(inferencefile)

In [15]:
def find_offsets(from_, to_, offsets):
    for arr in offsets:
        if arr[0] == from_ and arr[1] == to_:
            return arr
    return None

In [16]:

#store inference weights (calculation order and which idprmaps to add up per connection)
for i in xrange(len(t)):
    weights = t[i]
    child_joint = weights[0]
    parent_joint = weights[1]
    num_clusters = weights[2]
    offsets_1 = find_offsets(child_joint, parent_joint, out)
    offsets_2 = find_offsets(parent_joint, child_joint, out)
    print len(offsets_1[2]), weights[2]
    if offsets_1 == None or offsets_2 == None:
        print "Did not find offset between ", child_joint, " and ", parent_joint
    
    inf_weights[i,0:len(t[i])] = t[i]
    inf_weights[i,len(t[i]):len(t[i])+len(offsets_1[2])] = offsets_1[2]
    inf_weights[i,len(t[i])+len(offsets_1[2]):len(t[i])+len(offsets_1[2])+len(offsets_2[2])] = offsets_2[2]

20 10
20 10
12 6
12 6
6 3
6 3
20 10
20 10
12 6
12 6
6 3
6 3


In [17]:
print inf_weights[4]

[   4.            1.            3.           52.           53.           54.
   19.           20.           21.         -105.59290314  -25.98464775
 -135.61750793  -10.96753693  -72.58165741  -38.77911758   72.83905029
   38.72625351  105.9141922    25.74417305  135.87535095   10.92263603
    0.            0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.            0.
    0.           

In [18]:
# save changes to new binaryproto
net.save('../../models/swimmers_fullconv/swimmers_fullconv_35x35_new.protobuf')

3 0 1
3 0 7
3 1 0
10 1 2
3 1 4
10 2 1
10 2 3
10 3 2
3 4 1
6 4 5
6 5 4
6 5 6
6 6 5
3 7 0
10 7 8
3 7 10
10 8 7
10 8 9
10 9 8
3 10 7
6 10 11
6 11 10
6 11 12
6 12 11


In [118]:
print out[1]

[0, 7, [set([-9.20967013104, 56.0402169001]), set([17.5890585242, 25.381043257]), set([-3.87093596059, 38.3596059113])]]


In [119]:
len(out)

24